# 🧠 ERP TN Group - Vector Database Tutorial

**Author:** Lam Van Truyen  
**Email:** lamvantruyen@gmail.com  
**Website:** shareapiai.com  

Hướng dẫn sử dụng Vector Database với PostgreSQL + pgvector cho AI/ML applications.

## 📋 Setup và Import Libraries

In [ ]:
# Install required packages
!pip install psycopg2-binary numpy pandas scikit-learn openai sentence-transformers

import psycopg2
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import json
import os
from typing import List, Dict

## 🔌 Database Connection

In [ ]:
# Database connection parameters
DB_CONFIG = {
    'host': 'postgres_secure',
    'port': 5432,
    'database': 'erp_tngroup',
    'user': 'erp_admin',
    'password': 'TnGroup@2024!Secure#db$2025'  # Thay bằng password thực tế
}

def get_db_connection():
    """Establish database connection"""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("✅ Connected to ERP Vector Database")
        return conn
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return None

# Test connection
conn = get_db_connection()

## 🧠 Load Sentence Transformer Model

In [ ]:
# Load pre-trained sentence transformer
print("📥 Loading sentence transformer model...")
model = SentenceTransformer('all-MiniLM-L6-v2')  # 384 dimensions
print(f"✅ Model loaded - Embedding dimension: {model.get_sentence_embedding_dimension()}")

## 📄 Example 1: Document Embedding và Similarity Search

In [ ]:
# Sample documents
documents = [
    {
        'id': 'doc_001',
        'title': 'Introduction to Machine Learning',
        'content': 'Machine learning is a subset of artificial intelligence that focuses on algorithms and statistical models.'
    },
    {
        'id': 'doc_002', 
        'title': 'Deep Learning Fundamentals',
        'content': 'Deep learning uses neural networks with multiple layers to model and understand complex patterns.'
    },
    {
        'id': 'doc_003',
        'title': 'Vector Database Applications',
        'content': 'Vector databases enable efficient similarity search and are essential for AI applications like recommendation systems.'
    },
    {
        'id': 'doc_004',
        'title': 'PostgreSQL and pgvector',
        'content': 'PostgreSQL with pgvector extension provides powerful vector operations for machine learning applications.'
    }
]

# Generate embeddings
print("🔄 Generating embeddings for documents...")
for doc in documents:
    # Combine title and content for embedding
    text = f"{doc['title']} {doc['content']}"
    embedding = model.encode(text)
    doc['embedding'] = embedding.tolist()
    
print(f"✅ Generated embeddings for {len(documents)} documents")

In [ ]:
# Insert documents vào vector database
def insert_document(conn, doc_id, title, content, embedding):
    """Insert document embedding vào database"""
    try:
        with conn.cursor() as cur:
            # Note: Sử dụng 384 dimensions cho sentence-transformers
            cur.execute("""
                INSERT INTO vector_db.document_embeddings 
                (document_id, title, content, embedding, source)
                VALUES (%s, %s, %s, %s, %s)
                ON CONFLICT (document_id) DO UPDATE SET
                    title = EXCLUDED.title,
                    content = EXCLUDED.content,
                    embedding = EXCLUDED.embedding,
                    updated_at = CURRENT_TIMESTAMP
            """, (doc_id, title, content, embedding, 'jupyter_notebook'))
        conn.commit()
        return True
    except Exception as e:
        print(f"❌ Insert failed: {e}")
        conn.rollback()
        return False

# Insert all documents
if conn:
    for doc in documents:
        success = insert_document(conn, doc['id'], doc['title'], doc['content'], doc['embedding'])
        if success:
            print(f"✅ Inserted: {doc['title']}")
        else:
            print(f"❌ Failed: {doc['title']}")

In [ ]:
# Test similarity search
query_text = "artificial intelligence and neural networks"
query_embedding = model.encode(query_text).tolist()

print(f"🔍 Searching for: '{query_text}'")
print(f"📊 Query embedding dimension: {len(query_embedding)}")

if conn:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT 
                document_id,
                title,
                1 - (embedding <=> %s::vector) as similarity,
                LEFT(content, 100) || '...' as content_preview
            FROM vector_db.document_embeddings
            WHERE source = 'jupyter_notebook'
            ORDER BY embedding <=> %s::vector
            LIMIT 5
        """, (query_embedding, query_embedding))
        
        results = cur.fetchall()
        
        print("\n📋 Search Results:")
        for i, (doc_id, title, similarity, content) in enumerate(results, 1):
            print(f"{i}. {title}")
            print(f"   Similarity: {similarity:.3f}")
            print(f"   Content: {content}")
            print()

## 🛍️ Example 2: Product Recommendation System

In [ ]:
# Sample products
products = [
    {
        'id': 'prod_laptop_001',
        'name': 'Gaming Laptop RTX 4080',
        'description': 'High-performance gaming laptop with NVIDIA RTX 4080, Intel i9 processor, 32GB RAM, perfect for gaming and AI development',
        'category': 'Electronics',
        'price': 2499.99
    },
    {
        'id': 'prod_laptop_002',
        'name': 'MacBook Pro M3 Max',
        'description': 'Apple MacBook Pro with M3 Max chip, 36GB unified memory, ideal for machine learning and creative work',
        'category': 'Electronics', 
        'price': 3999.99
    },
    {
        'id': 'prod_phone_001',
        'name': 'iPhone 15 Pro Max',
        'description': 'Latest iPhone with A17 Pro chip, advanced camera system, titanium design',
        'category': 'Electronics',
        'price': 1199.99
    },
    {
        'id': 'prod_headphone_001',
        'name': 'Sony WH-1000XM5',
        'description': 'Premium noise-canceling headphones with industry-leading sound quality',
        'category': 'Electronics',
        'price': 399.99
    },
    {
        'id': 'prod_book_001',
        'name': 'Hands-On Machine Learning',
        'description': 'Comprehensive guide to machine learning with Python, TensorFlow, and scikit-learn',
        'category': 'Books',
        'price': 59.99
    }
]

# Generate product embeddings
print("🔄 Generating product embeddings...")
for product in products:
    # Combine name and description
    text = f"{product['name']} {product['description']} {product['category']}"
    embedding = model.encode(text)
    product['embedding'] = embedding.tolist()
    
print(f"✅ Generated embeddings for {len(products)} products")

In [ ]:
# Insert products vào database
def insert_product(conn, product_id, name, description, category, price, embedding):
    """Insert product embedding vào database"""
    try:
        with conn.cursor() as cur:
            cur.execute("""
                INSERT INTO vector_db.product_embeddings 
                (product_id, product_name, description, category, price, embedding)
                VALUES (%s, %s, %s, %s, %s, %s)
                ON CONFLICT (product_id) DO UPDATE SET
                    product_name = EXCLUDED.product_name,
                    description = EXCLUDED.description,
                    category = EXCLUDED.category,
                    price = EXCLUDED.price,
                    embedding = EXCLUDED.embedding,
                    updated_at = CURRENT_TIMESTAMP
            """, (product_id, name, description, category, price, embedding))
        conn.commit()
        return True
    except Exception as e:
        print(f"❌ Insert failed: {e}")
        conn.rollback()
        return False

# Insert all products
if conn:
    for product in products:
        success = insert_product(
            conn, product['id'], product['name'], product['description'], 
            product['category'], product['price'], product['embedding']
        )
        if success:
            print(f"✅ Inserted: {product['name']}")

In [ ]:
# Test product recommendations
reference_product = 'prod_laptop_001'  # Gaming Laptop

print(f"🛍️ Getting recommendations for: {reference_product}")

if conn:
    with conn.cursor() as cur:
        # Get similar products
        cur.execute("""
            SELECT 
                pe2.product_id,
                pe2.product_name,
                pe2.category,
                pe2.price,
                1 - (pe1.embedding <=> pe2.embedding) as similarity
            FROM vector_db.product_embeddings pe1
            CROSS JOIN vector_db.product_embeddings pe2
            WHERE pe1.product_id = %s 
              AND pe2.product_id != %s
            ORDER BY pe1.embedding <=> pe2.embedding
            LIMIT 5
        """, (reference_product, reference_product))
        
        results = cur.fetchall()
        
        print("\n📋 Product Recommendations:")
        for i, (prod_id, name, category, price, similarity) in enumerate(results, 1):
            print(f"{i}. {name}")
            print(f"   Category: {category}")
            print(f"   Price: ${price:.2f}")
            print(f"   Similarity: {similarity:.3f}")
            print()

## 📊 Vector Database Analytics

In [ ]:
# Get database statistics
if conn:
    print("📊 Vector Database Statistics")
    print("=" * 40)
    
    with conn.cursor() as cur:
        # Document stats
        cur.execute("SELECT * FROM vector_db.document_stats")
        doc_stats = cur.fetchone()
        
        if doc_stats:
            print(f"📄 Documents:")
            print(f"   Total: {doc_stats[0]}")
            print(f"   Sources: {doc_stats[1]}")
            print(f"   Avg Dimensions: {doc_stats[2]}")
            print()
        
        # Product stats
        cur.execute("SELECT * FROM vector_db.product_stats")
        prod_stats = cur.fetchone()
        
        if prod_stats:
            print(f"🛍️ Products:")
            print(f"   Total: {prod_stats[0]}")
            print(f"   Categories: {prod_stats[1]}")
            print(f"   Avg Price: ${prod_stats[2]:.2f}" if prod_stats[2] else "N/A")
            print(f"   Active: {prod_stats[3]}")
            print()
        
        # Index information
        cur.execute("""
            SELECT indexname, tablename 
            FROM pg_indexes 
            WHERE schemaname = 'vector_db' AND indexname LIKE '%vector%'
        """)
        indexes = cur.fetchall()
        
        print(f"🔍 Vector Indexes:")
        for idx_name, table_name in indexes:
            print(f"   {idx_name} -> {table_name}")

## 🚀 Next Steps

### 🔮 Advanced Usage:
1. **OpenAI Integration** - Use OpenAI embeddings for better quality
2. **Hybrid Search** - Combine text search với vector similarity
3. **Real-time Updates** - Stream embeddings từ applications
4. **Performance Tuning** - Optimize IVFFLAT indexes
5. **A/B Testing** - Compare different embedding models

### 📚 Resources:
- **pgvector Documentation**: https://github.com/pgvector/pgvector
- **Sentence Transformers**: https://www.sbert.net/
- **OpenAI Embeddings**: https://platform.openai.com/docs/guides/embeddings
- **PostgreSQL Performance**: https://www.postgresql.org/docs/current/performance-tips.html

### 👨‍💻 Developer:
**Lam Van Truyen**  
📧 lamvantruyen@gmail.com  
🌐 shareapiai.com  
💼 https://www.linkedin.com/in/lamtruyen/

In [ ]:
# Cleanup - Close database connection
if conn:
    conn.close()
    print("🔌 Database connection closed")
    
print("\n🎉 Vector Database tutorial completed!")
print("📧 Questions? Contact: lamvantruyen@gmail.com")